In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import joblib # Import joblib directly instead of from sklearn.externals

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Path to the main directory containing your folders
main_folder_path = '/content/drive/MyDrive/Dataset2'

# List all folders in the main directory
folders = [f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))]
print(folders)  # This will print the names of your folders

['Severe DR', 'Proliferate DR', 'Mild DR', 'Moderate DR', 'Healthy']


In [ ]:
import os

# Path to your main directory in Google Drive
path_data = '/content/drive/MyDrive/Dataset2'  # Update this path

# List the main folder contents
data = os.listdir(path_data)

# List each category folder's contents
healthy = os.listdir(os.path.join(path_data, 'Healthy'))
mild = os.listdir(os.path.join(path_data, 'Mild DR'))
moderate = os.listdir(os.path.join(path_data, 'Moderate DR'))
proliferate = os.listdir(os.path.join(path_data, 'Proliferate DR'))
severe = os.listdir(os.path.join(path_data, 'Severe DR'))

# Optional: Print the contents for verification
print("Healthy images:", healthy)
print("Mild DR images:", mild)
print("Moderate DR images:", moderate)
print("Proliferate DR images:", proliferate)
print("Severe DR images:", severe)


Healthy images: ['Healthy_476.png', 'Healthy_473.png', 'Healthy_474.png', 'Healthy_472.png', 'Healthy_475.png', 'Healthy_469.png', 'Healthy_478.png', 'Healthy_479.png', 'Healthy_484.png', 'Healthy_490.png', 'Healthy_493.png', 'Healthy_471.png', 'Healthy_487.png', 'Healthy_495.png', 'Healthy_468.png', 'Healthy_486.png', 'Healthy_483.png', 'Healthy_520.png', 'Healthy_517.png', 'Healthy_481.png', 'Healthy_4.png', 'Healthy_3.png', 'Healthy_555.png', 'Healthy_22.png', 'Healthy_2.png', 'Healthy_21.png', 'Healthy_20.png', 'Healthy_18.png', 'Healthy_19.png', 'Healthy_17.png', 'Healthy_16.png', 'Healthy_17(1).png', 'Healthy_15.png', 'Healthy_775.png', 'Healthy_798.png', 'Healthy_783.png', 'Healthy_787.png', 'Healthy_773.png', 'Healthy_800.png', 'Healthy_777.png']
Mild DR images: ['Mild_DR_250.png', 'Mild_DR_260.png', 'Mild_DR_263.png', 'Mild_DR_244.png', 'Mild_DR_257.png', 'Mild_DR_255.png', 'Mild_DR_266.png', 'Mild_DR_276.png', 'Mild_DR_261.png', 'Mild_DR_247.png', 'Mild_DR_251.png', 'Mild_DR_

In [ ]:
print("classes names :", (data), "\n______________________________\n")
print("Number of classes :", len(data), "\n______________________________\n")
print("Number of Healty images :", len(healthy), "\n______________________________\n")
print("Number of Mild images :", len(mild),  "\n______________________________\n")
print("Number of Moderate images :", len(moderate),  "\n______________________________\n")
print("Number of Proliferate images :", len(proliferate),  "\n______________________________\n")
print("Number of severe images :", len(severe),  "\n______________________________\n")

classes names : ['Severe DR', 'Proliferate DR', 'Mild DR', 'Moderate DR', 'Healthy'] 
______________________________

Number of classes : 5 
______________________________

Number of Healty images : 40 
______________________________

Number of Mild images : 57 
______________________________

Number of Moderate images : 71 
______________________________

Number of Proliferate images : 79 
______________________________

Number of severe images : 48 
______________________________



In [ ]:
main_folder_path = '/content/drive/MyDrive/Dataset2'
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
)

# Load data
train_data_gen = datagen.flow_from_directory(
    main_folder_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_data_gen = datagen.flow_from_directory(
    main_folder_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)
print("Class labels:", train_data_gen.class_indices)

Found 238 images belonging to 5 classes.
Found 57 images belonging to 5 classes.
Class labels: {'Healthy': 0, 'Mild DR': 1, 'Moderate DR': 2, 'Proliferate DR': 3, 'Severe DR': 4}


In [ ]:
from tensorflow.keras import layers, models

def create_cnn(input_shape=(128, 128, 3), num_classes=5):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train CNN Model
cnn_model = create_cnn()
cnn_model.fit(train_data_gen, validation_data=val_data_gen, epochs=10)
cnn_model.save('cnn_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.2272 - loss: 1.7727 - val_accuracy: 0.2632 - val_loss: 1.5889
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.2526 - loss: 1.6003 - val_accuracy: 0.2456 - val_loss: 1.5826
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 901ms/step - accuracy: 0.1865 - loss: 1.5813 - val_accuracy: 0.2632 - val_loss: 1.5498
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.2697 - loss: 1.5440 - val_accuracy: 0.3509 - val_loss: 1.4399
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 18s 961ms/step - accuracy: 0.3195 - loss: 1.4750 - val_accuracy: 0.3333 - val_loss: 1.4121
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.3444 - loss: 1.4184 - val_accuracy: 0.2982 - val_loss: 1.4856
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3719 - loss: 1.4366 - val_accuracy: 0.2632 - val_loss: 1.4225
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 896ms/step - accuracy: 0.2950 - loss: 1.4827 - val_accuracy: 0.2982 - val_loss: 1.4165
Epoch 9/10

In [ ]:
# Extract features for SVM training
X_train_features = []
y_train_labels = []

for images, labels in train_data_gen:
    features = cnn_model.predict(images)
    X_train_features.extend(features)
    y_train_labels.extend(labels)
    if len(X_train_features) >= train_data_gen.samples:
        break

# Train SVM
svm_model = svm.SVC(kernel='linear', probability=True)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_features)
svm_model.fit(X_train_scaled, np.argmax(y_train_labels, axis=1))

# Save SVM and scaler
joblib.dump(svm_model, 'svm_model.pkl')
joblib.dump(scaler, 'scaler.pkl')


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step


['scaler.pkl']

In [ ]:
def ensemble_predict(image, cnn_model, svm_model, scaler):
    cnn_pred = cnn_model.predict(image)[0]
    cnn_class = np.argmax(cnn_pred)

    # Flatten and scale image for SVM
    image_flatten = image.flatten().reshape(1, -1)
    image_scaled = scaler.transform(image_flatten)
    svm_pred = svm_model.predict_proba(image_scaled)[0]
    svm_class = np.argmax(svm_pred)

    # Combine predictions (majority voting)
    combined_pred = cnn_pred + svm_pred
    final_class = np.argmax(combined_pred)

    classes = ['Healthy', 'Mild DR', 'Moderate DR', 'Proliferate DR', 'Severe DR']
    return classes[final_class]


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
import joblib
from PIL import Image

# Load your pre-trained models and scaler
cnn_model = load_model('cnn_model.h5')  # Load CNN model
svm_model = joblib.load('svm_model.pkl')  # Load SVM model
scaler = joblib.load('scaler.pkl')  # Load scaler for SVM preprocessing

# Define classes for diabetic retinopathy stages
classes = ['Healthy', 'Mild DR', 'Moderate DR', 'Proliferate DR', 'Severe DR']

# Define the ensemble prediction function
def ensemble_predict(img):
    # Preprocess image for CNN
    img_resized = img.resize((128, 128))
    img_array = np.array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict using CNN
    cnn_pred = cnn_model.predict(img_array)[0]

    # Preprocess image for SVM
    img_flatten = img_array.flatten().reshape(1, -1)
    img_scaled = scaler.transform(img_flatten)

    # Predict using SVM
    svm_pred = svm_model.predict_proba(img_scaled)[0]

    # Combine predictions using a weighted average (example weights: 0.6 CNN, 0.4 SVM)
    combined_pred = 0.6 * cnn_pred + 0.4 * svm_pred
    final_class = np.argmax(combined_pred)

    # Return the final class
    return f"Predicted Stage: {classes[final_class]}"

# Create Gradio interface
iface = gr.Interface(
    fn=ensemble_predict,  # Prediction function
    inputs=gr.Image(type="pil"),  # Image input
    outputs="text",  # Text output (class label)
    title="Diabetic Retinopathy Detection App",
    description="Welcome to our Diabetic Retinopathy Detection App! This app utilizes deep learning models to detect diabetic retinopathy in retinal images. Diabetic retinopathy is a common complication of diabetes and early detection is crucial for effective treatment.",
    examples=[
        "/content/drive/MyDrive/Dataset2/Healthy/Healthy_15.png",
        "/content/drive/MyDrive/Dataset2/Mild DR/Mild_DR_221.png",
        "/content/drive/MyDrive/Dataset2/Moderate DR/Moderate_DR_358.png",
        "/content/drive/MyDrive/Dataset2/Severe DR/Severe DR_10.png",
    ],
)

# Launch the Gradio interface
if __name__ == "__main__":
    iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fda0b029d483159fbc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


https://bhimrazy-diabetic-retinopathy-detection.hf.space